# XGBoost

### 1. Introdução ao XGBoost

### 2. Treino de Modelos

In [6]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from optuna import create_study
from optuna.pruners import MedianPruner
from optuna.exceptions import TrialPruned
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../dataset_processing/xtb_dataset.csv')
df

,Dipole,E_HOMO,E_LUMO,gap_HOMO-LUMO,ZPE,H,U,U0,G,Delta
0,0.727,-10.6203,-2.7950,-7.8253,0.154214,-27.917739,-27.918683,-28.080950,-27.959630,-12.397982
1,1.905,-10.4336,-2.2949,-8.1387,0.199866,-28.104404,-28.105348,-28.313524,-28.146070,-12.165407
2,4.475,-10.5746,-6.9494,-3.6252,0.120064,-27.272560,-27.273504,-27.400818,-27.312535,-13.078113
3,2.100,-9.8173,-5.5314,-4.2859,0.177677,-27.068399,-27.069343,-27.254803,-27.108720,-13.224129
4,0.872,-9.9722,-6.6172,-3.3550,0.133244,-24.148876,-24.149821,-24.292155,-24.192788,-16.186776
...,...,...,...,...,...,...,...,...,...,...
129152,4.710,-11.3840,-7.8335,-3.5505,0.121817,-26.808903,-26.809847,-26.939965,-26.851229,-13.538966
129153,4.996,-10.8793,-7.1097,-3.7696,0.141742,-28.278349,-28.279293,-28.428099,-28.317523,-12.050832
129154,3.973,-10.9497,-7.7346,-3.2151,0.124894,-24.819326,-24.820270,-24.952019,-24.858199,-15.526912
129155,0.918,-10.2869,-1.8611,-8.4258,0.195977,-30.002576,-30.003520,-30.208931,-30.047056,-10.270000


In [3]:
X = df.drop("Delta", axis=1)
y = df["Delta"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=27)

In [7]:
def suggest_xgb_params(trial):
    params = {}

    params['booster'] = trial.suggest_categorical('booster', ['gbtree', 'dart'])
    params['objective'] = 'reg:squarederror'
    params['verbosity'] = 0
    params['random_state'] = trial.number
    params['n_jobs'] = 1

    params['learning_rate'] = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)

    params['n_estimators'] = trial.suggest_int('n_estimators', 100, 3000)  # usar early stopping/pruner

    params['subsample'] = trial.suggest_float('subsample', 0.5, 1.0)
    params['colsample_bytree'] = trial.suggest_float('colsample_bytree', 0.4, 1.0)
    params['colsample_bylevel'] = trial.suggest_float('colsample_bylevel', 0.4, 1.0)
    params['colsample_bynode'] = trial.suggest_float('colsample_bynode', 0.4, 1.0)

    params['min_child_weight'] = trial.suggest_float('min_child_weight', 1e-3, 10.0, log=True)
    params['gamma'] = trial.suggest_float('gamma', 1e-8, 10.0, log=True)

    params['reg_alpha'] = trial.suggest_float('reg_alpha', 1e-8, 100.0, log=True)
    params['reg_lambda'] = trial.suggest_float('reg_lambda', 1e-8, 100.0, log=True)

    params['max_delta_step'] = trial.suggest_int('max_delta_step', 0, 10)

    params['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
    if params['grow_policy'] == 'depthwise':
        params['max_depth'] = trial.suggest_int('max_depth', 3, 12)
        # max_leaves is ignored for depthwise, ensure not set (or set large)
        params['max_leaves'] = 0
    else:  # lossguide
        params['max_depth'] = 0  # ignored by lossguide
        params['max_leaves'] = trial.suggest_int('max_leaves', 8, 1024)

    if params['booster'] == 'dart':
        params['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        params['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        params['rate_drop'] = trial.suggest_float('rate_drop', 0.0, 0.3)
        params['skip_drop'] = trial.suggest_float('skip_drop', 0.0, 0.3)
    else:
        # ensure DART params not present for gbtree (optional: they are simply ignored)
        params['sample_type'] = 'uniform'
        params['normalize_type'] = 'tree'
        params['rate_drop'] = 0.0
        params['skip_drop'] = 0.0

    return params

In [ ]:
def inst_xgboost(trial):
    parametros = suggest_xgb_params(trial)

    normalizar = trial.suggest_categorical('normalizar', [None, 'stds', 'pca', 'limiar', 'rfe'])
    if normalizar == 'stds':
        modelo = make_pipeline(StandardScaler(), XGBRegressor(**parametros))
    elif normalizar == "pca":
        # Definindo o número de componentes a serem mantidas
        components = trial.suggest_int("pca_components", 2, 9)
        modelo = make_pipeline(
            StandardScaler(),
            PCA(components),
            XGBRegressor(**parametros))
    elif normalizar == "limiar":
        # Definindo o Limiar a ser utilizado
        threshold = trial.suggest_float("variance_threshold", 0, 0.1)
        modelo = make_pipeline(
            StandardScaler(),
            VarianceThreshold(threshold),
            XGBRegressor(**parametros)
        )
    elif normalizar == "rfe": 
        # Criando o modelo para parametro do RFE
        estimator = RandomForestRegressor()
        # Definindo o número de atributos a serem mantidos
        n_features_to_select = trial.suggest_int("rfe_features", 2, 9)
        modelo = make_pipeline(
            StandardScaler(),
            RFE(estimator=estimator, n_features_to_select=n_features_to_select),
            XGBRegressor(**parametros)
        )
    else:
        modelo = XGBRegressor(**parametros)

    return modelo

In [10]:
def funcao_objetivo(trial, X, y, num_folds, instanciador, random_state=42):
    X_arr = np.asarray(X)
    y_arr = np.asarray(y)

    modelo = instanciador(trial)

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    fold_scores = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_arr)):
        X_train, X_val = X_arr[train_idx], X_arr[val_idx]
        y_train, y_val = y_arr[train_idx], y_arr[val_idx]

        modelo.fit(X_train, y_train)
        preds = modelo.predict(X_val)

        # Compatível com todas as versões do sklearn:
        mse = mean_squared_error(y_val, preds)
        rmse = float(np.sqrt(mse))
        fold_scores.append(rmse)

        trial.report(rmse, step=fold_idx)
        if trial.should_prune():
            raise TrialPruned()

    return float(np.mean(fold_scores))


In [11]:
def rodar_optuna(nome_estudo, X, y, instanciador, num_folds=5, n_trials=1000):
    study = create_study(
        direction='minimize',
        study_name=nome_estudo,
        storage=f'sqlite:///{nome_estudo}.db',
        load_if_exists=True,
        pruner=MedianPruner()
    )

    def objetivo_parcial(trial):
        return funcao_objetivo(trial, X, y, num_folds, instanciador)

    study.optimize(objetivo_parcial, n_trials=n_trials, show_progress_bar=True, n_jobs=3)
    return study



In [ ]:
study_en = rodar_optuna('elas_xtb', X_train, y_train, inst_xgboost)